In [ ]:
# Install the newer OpenAI SDK
!pip install --upgrade openai

from openai import OpenAI
import pandas as pd
import random
import re

In [ ]:
client = OpenAI(api_key="KEY_GOES_HERE")

def query_chatGPT(prompt):
    """Send a prompt to OpenAI GPT-4o."""
    response = client.chat.completions.create(
        model="gpt-4o-mini",       # you can change to gpt-4o if you want
        messages=[{"role": "user", "content": prompt}],
        max_tokens=400,
        temperature=1
    )
    return response.choices[0].message.content

In [ ]:
# download public domain poems
poems_df = pd.read_csv(
    'https://github.com/maria-antoniak/poetry-eval/'
    'raw/refs/heads/main/data/poetry-evaluation_public-domain-poems.csv'
)

# keep sonnets & ghazals
poems_df = poems_df[poems_df['form'].isin(['sonnet', 'ghazal'])]

# random 25 sonnets + 2 ghazals
sample_df = poems_df[poems_df['form'] == 'sonnet'].sample(25)
sample_df = pd.concat([sample_df, poems_df[poems_df['form'] == 'ghazal']])

# load brown poems
import gdown
url = "https://drive.google.com/file/d/1SqTw3A99hPDbTUxZ-OoJD4wGdl0ZF3Oz/view?usp=sharing"
output = "brown_poems.csv"
gdown.download(url=url, output=output, fuzzy=True)

brown_df = pd.read_csv("brown_poems.csv")

# reduce sample_df columns
sample_df = sample_df[['author', 'poem_title', 'poem_text', 'form']]

# combine datasets
combined_df = pd.concat([sample_df, brown_df])

Downloading...
From: https://drive.google.com/uc?id=1SqTw3A99hPDbTUxZ-OoJD4wGdl0ZF3Oz
To: /content/brown_poems.csv
100%|██████████| 3.18k/3.18k [00:00<00:00, 8.18MB/s]


In [ ]:
prompt_template_start = '''
Read the following poem and classify its form as: [sonnet, ghazal, duplex].
You must choose exactly ONE.

Return the answer in this format:

1. Poetic Form: ...
2. Elaborated Rationale: ...
3. One-Word Summary: ...
4. Confidence Score: ...

Poem Text:
'''

prompt_template_end = '''

Pick ONE of these forms: [sonnet, ghazal, duplex].
'''

In [ ]:
def extract_response_fields(response):
    """Parse the classification output."""
    lines = response.strip().split('\n')

    result = {
        'poetic_form': None,
        'rationale': None,
        'summary': None,
        'confidence': None
    }

    for line in lines:
        if line.startswith('1. Poetic Form:'):
            result['poetic_form'] = line.split(':', 1)[1].strip()
        elif line.startswith('2. Elaborated Rationale:'):
            result['rationale'] = line.split(':', 1)[1].strip()
        elif line.startswith('3. One-Word Summary:'):
            result['summary'] = line.split(':', 1)[1].strip()
        elif line.startswith('4. Confidence Score:'):
            result['confidence'] = line.split(':', 1)[1].strip()

    return result

In [ ]:
for index, row in combined_df.iterrows():
    prompt = prompt_template_start + row['poem_text'] + prompt_template_end
    response = query_chatGPT(prompt)
    fields = extract_response_fields(response)

    combined_df.at[index, 'poetic_form'] = fields['poetic_form']
    combined_df.at[index, 'rationale'] = fields['rationale']
    combined_df.at[index, 'summary'] = fields['summary']
    combined_df.at[index, 'confidence'] = fields['confidence']

combined_df.to_csv("combined_df.csv", index=False)

In [ ]:
subjects = [
    "abuse",
    "activities",
    "arts & sciences",
    "christianity",
    "chronic illness",
    "crime & punishment",
    "cycles",
    "desire & passion",
    "doubt & contemplation",
    "family & ancestors",
    "gender & sexuality",
    "greek & roman mythology",
    "history & politics",
    "home life",
    "hope",
    "humor & satire",
    "indoor activities",
    "life choices",
    "lgbtq+",
    "living",
    "love",
    "love death avoidance",
    "memory & nostalgia",
    "men & women",
    "mythology & folklore",
    "nature",
    "power",
    "poetry & poets",
    "race & ethnicity",
    "relationships",
    "romantic love",
    "sexual violence",
    "sorrow & grieving",
    "social commentaries",
    "subjugation",
    "the mind",
    "trees and flowers",
    "truth",
    "twilight",
    "urban environment",
    "war & conflict",
    "weather",
    "youth"
]


def generate_general_prompt(subject, form="Duplex"):
    return f"Write a poem about the subject of {subject} in the form: {form}."


def generate_figurative_prompt(subject, form="Duplex"):
    return f"Write a poem about the subject of {subject} in the form: {form}. Do not use the words {subject} or {form}."


def generate_specific_prompt(subject, form="Duplex"):
    return f"Write a poem about the subject of {subject} in the form: {form}. Make it about something specific."


new_poems_df = pd.DataFrame(columns=['subject', 'form', 'prompt_type', 'poem_text'])

for subject in subjects:
    genPrompt = generate_general_prompt(subject)
    figPrompt = generate_figurative_prompt(subject)
    specPrompt = generate_specific_prompt(subject)

    genResponse = query_chatGPT(genPrompt)
    figResponse = query_chatGPT(figPrompt)
    specResponse = query_chatGPT(specPrompt)

    new_poems_df.loc[len(new_poems_df)] = [subject, "Duplex", "general", genResponse]
    new_poems_df.loc[len(new_poems_df)] = [subject, "Duplex", "figurative", figResponse]
    new_poems_df.loc[len(new_poems_df)] = [subject, "Duplex", "specific", specResponse]

new_poems_df.to_csv("new_poems_df.csv", index=False)

print("All tasks complete.")

All tasks complete.


In [ ]:
BROWN_RULES = """
Write a poem in the Duplex form as defined by Jericho Brown.
You MUST follow ALL of these rules:

1. The poem has exactly 14 lines.
2. Each line must contain between 9 and 11 syllables.
3. The poem should blend qualities of ghazal, sonnet, and blues traditions.
4. Line 1 is repeated as Line 14.
5. Line 2 must reinterpret Line 1 in an unexpected way.
6. Line 2 is repeated as Line 3.
7. Line 4 must reinterpret Line 3 in an unexpected way.
8. This pattern of echo / reinterpretation continues until Line 13.
9. Line 13 becomes the first line of the couplet that leads to the final line (Line 14).
10. The poem’s theme should be rooted in emotional tension, reflection, or personal revelation.
"""

In [ ]:
def generate_general_prompt(subject, form="Duplex"):
    return (
        f"{BROWN_RULES}\n\n"
        f"Write a poem about the subject of {subject} in the form: {form}.\n"
        f"The poem MUST follow all Duplex rules above."
    )


def generate_figurative_prompt(subject, form="Duplex"):
    return (
        f"{BROWN_RULES}\n\n"
        f"Write a poem about the subject of {subject} in the form: {form}.\n"
        f"Do NOT use the words '{subject}' or '{form}' anywhere in the poem.\n"
        f"The poem MUST follow all Duplex rules above."
    )


def generate_specific_prompt(subject, form="Duplex"):
    return (
        f"{BROWN_RULES}\n\n"
        f"Write a poem about the subject of {subject} in the form: {form}.\n"
        f"Make the poem about a very specific moment, object, place, or memory.\n"
        f"The poem MUST follow all Duplex rules above."
    )

In [ ]:
new_brown_poems_df = pd.DataFrame(
    columns=['subject', 'form', 'prompt_type', 'poem_text']
)

In [ ]:
for subject in subjects:

    # build 3 prompt types
    genPrompt = generate_general_prompt(subject)
    figPrompt = generate_figurative_prompt(subject)
    specPrompt = generate_specific_prompt(subject)

    # call your model function (this is unchanged)
    genResponse = query_chatGPT(genPrompt)
    figResponse = query_chatGPT(figPrompt)
    specResponse = query_chatGPT(specPrompt)

    # store in your preferred format
    new_brown_poems_df.loc[len(new_brown_poems_df)] = [
        subject, "Duplex", "general", genResponse
    ]
    new_brown_poems_df.loc[len(new_brown_poems_df)] = [
        subject, "Duplex", "figurative", figResponse
    ]
    new_brown_poems_df.loc[len(new_brown_poems_df)] = [
        subject, "Duplex", "specific", specResponse
    ]

In [ ]:
new_brown_poems_df.to_csv("new_brown_poems_df.csv", index=False)

print("All Duplex poems generated and saved.")

All Duplex poems generated and saved.
